In [0]:
%run ./invoice_mapping

**final**

In [0]:
# df_price.filter(col("product_code") == "BEK455310M").display()

In [0]:
df_union = df_month_joined.union(df_week_joined)
# display(df_union)

In [0]:
df_month_count = df_month_joined.count()
display(df_month_count)

In [0]:
df_week_count = df_week_joined.count()
display(df_week_count)

In [0]:
df_union_count = df_union.count()
display(df_union_count)

In [0]:
df_dup_rm = df_union.dropDuplicates()
df_dup_rm_count = df_dup_rm.count()
display(df_dup_rm_count)

In [0]:
df_union_select = df_union.select("product_code", "year", "month", "sales_amount")

df_union_group = df_union_select.groupBy("product_code", "year", "month","sales_amount").count().orderBy("product_code","year","month")

# display(df_union_group)

**previous month sales amount**

In [0]:
window = (
    Window.partitionBy("product_code")
    .orderBy(col("year"), col("month"))
    .rowsBetween(Window.unboundedPreceding, Window.currentRow)
)

price_recent = df_union.withColumn("recent_sales_amount",
    last(
        when(col("sales_amount").isNotNull() & (col("sales_amount") != 0), col("sales_amount")),
        ignorenulls=True
    ).over(window)
)

# price_recent.display()

In [0]:
price_filled = price_recent.withColumn(
    "sales_amount_new",
    when(col("sales_amount").isNull(), col("recent_sales_amount"))
    .when(col("sales_amount") == 0, col("sales_amount"))
    .otherwise(col("sales_amount"))
)

# display(price_filled)


In [0]:
df = price_filled.drop("recent_sales_amount", "sales_amount","price_date")\
                    .withColumnRenamed("sales_amount_new", "sales_amount")
# display(df)

In [0]:
price_filled.select("product_code", "year", "month", "sales_amount", "recent_sales_amount", "sales_amount_new")

df_sorted = price_filled.groupBy("product_code", "year", "month","sales_amount","recent_sales_amount", "sales_amount_new").count().orderBy("product_code","year","month")

display(df_sorted )

In [0]:
df.filter(col("product_code") == "BPK556360M").display()